###  Planet Analytics API Tutorial

<h1 style="margin-top:10px;">Case Study: Flood and Displacement Mapping in Syria</h1>
</div>
<div class="content-block">

## Overview

1. [Setup](#1.-Setup)

2. [Case Study](#2.-Case-Study)

3. [Mapping our Area of Interest](#3.-Mapping-AOI) 

4. [Working with Collections](#4.-Working-with-Collections)
    
5. [Parsing Results](#5.-Parse-Results-Links)
    
6. [Segmentation Results](#6.-Segmentation-Results)
    
7. [Quantifying Change](#7.-Quantifying-Change)
    
8. [Line Charts](#8.-Line-Charts)
    
</div>

## 1. Setup
To run through this notebook, you will need access to the following:
- A Planet account and Planet API Key
- Access to the Analytics API

In [ ]:
import os

# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = 'PASTE YOUR API KEY HERE'

    # construct auth tuple for use in the requests library
BASIC_AUTH = (API_KEY, '')

**Set the base url for the Planet Analytic Feeds product**

See the [Analytics API Docs](https://developers.planet.com/docs/analytics/) for more details.

In [ ]:
BASE_URL = "https://api.planet.com/analytics/"

Check API Connection

In [ ]:
import requests

feed_list_url = BASE_URL + 'feeds'
resp = requests.get(feed_list_url, auth=BASIC_AUTH, params={'limit': 1})
if resp.status_code == 200:
    print('Yay, you can access the Analytics API')
else:
    print('Something is wrong:', resp.content)

## 2. Case Study

Near the end of 2018, Syria experienced severe rainfall and flooding across much of the Northern latitudes of the country. This flooding had devastating impacts to several Internally Displaced Persons (IDP) camps  across the region, as [reported](https://www.savethechildren.net/news/northern-syria-flooding-thousands-children-risk-further-displacement)  in [several](https://reliefweb.int/report/syrian-arab-republic/record-rainfall-devastates-idp-camps-northern-syria) small media [outlets](https://arab24.com/portal/index.php/arab24-stories/syria/item/12613-2018-12-07-15-17-41). 

Today, we are interested using Planet's Analytic Feeds to explore both the development of an IDP Camp south of Al Hasakah, and the impacts of the flooding of this area in subsequent years. We will use the following buildings and roads suscription IDs to collect our data.

**Note:** If you do not have access to these subscriptions, please [get in touch](https://www.planet.com/contact-sales/).

In [ ]:
buildings_sub_id = '76d06ec1-8507-4035-97cd-b3ea87b5b699'
roads_sub_id = '6696da5c-88b8-49c2-a423-c936c0f386a5'

Let's create a new url to request the subscriptions endpoint.

In [ ]:
subscriptions_url = BASE_URL + 'subscriptions/'

syria_buildings = requests.get(subscriptions_url + buildings_sub_id, auth=BASIC_AUTH).json()
syria_roads = requests.get(subscriptions_url + roads_sub_id, auth=BASIC_AUTH) .json()

We can use the `pprint` library to structure our json responses. 

In [ ]:
from pprint import pprint

pprint(syria_buildings)
print('')
pprint(syria_roads)

## 3. Mapping AOI

**Inspecting subscription details**

Subscriptions have a spatial area of interest described by a geojson geometry. We can visualize the area of interest for a subscription on a map. First, let's just confirm that the geometries are the same for both roads and buildings subscriptions.

In [ ]:
if syria_buildings['geometry'] == syria_roads['geometry']:
    print('The geometries are the same!')
    aoi = syria_buildings['geometry']

In [ ]:
from ipyleaflet import Map, GeoJSON, LocalTileLayer, LayersControl, SplitMapControl, WidgetControl
from ipywidgets import SelectionSlider, VBox
from numpy import mean

First, let's center the map at the centroid of our geometry.

In [ ]:
aoi

In [ ]:
min_lat = min(coord[1] for coord in aoi['coordinates'][0])
max_lat = max(coord[1] for coord in aoi['coordinates'][0])
min_lon = min(coord[0] for coord in aoi['coordinates'][0])
max_lon = max(coord[0] for coord in aoi['coordinates'][0])

map_center = (mean([min_lat, max_lat]), mean([min_lon, max_lon]))
print(map_center)

In [ ]:
# make a map, and draw the subscription geometry
m = Map(center=map_center, zoom=12)

# convert to leaflet GeoJSON object
map_AOI = GeoJSON(
    data = aoi, 
    style = {'color': 'blue', 'opacity':0.5, 'weight':1.5, 'dashArray':'5', 'fillOpacity':0.1}
)

m.add_layer(map_AOI)
    
m

Now that we have a sense of our study AOI, let's inspect Planet's source imagery and Analytic Feeds results.

## 4. Working with Collections

As should now be familiar, Planet Analytic Feeds results can be accessed via the `collections` endpoint. We can find the results for our particular subscriptions in the `links` property from the results of our last `subscriptions` requests.

In [ ]:
buildings_collection_endpoint = [link['href'] for link in syria_buildings['links'] if link['rel'] == 'results'][0]
roads_collection_endpoint = [link['href'] for link in syria_roads['links'] if link['rel'] == 'results'][0]

print('Buildings Collection URL: {}'.format(buildings_collection_endpoint))
print('Roads Collection URL: {}'.format(roads_collection_endpoint))

**Request the Collections API**

In [ ]:
building_results = requests.get(buildings_collection_endpoint, auth=BASIC_AUTH).json()
roads_results = requests.get(roads_collection_endpoint, auth=BASIC_AUTH).json()

print('We got {} buildings results!'.format(len(building_results['features'])))
print('We got {} roads results!'.format(len(roads_results['features'])))

## 5. Parse Results Links

Our results come back nicely wrapped as GeoJSON FeatureCollection. We can easily put them into a Pandas Dataframe.

In [ ]:
import pandas as pd

**Parse Buildings Results**

In [ ]:
# Convert links and properties to dataframe
buildings = pd.json_normalize(building_results['features']).loc[:, ['links', 'properties.observed']]

# extract links 
buildings['source_tiles'] = buildings['links'].map(lambda links: [link['href'] for link in links if link['rel'] == 'source-tiles'][0])
buildings['buildings_tiles'] = buildings['links'].map(lambda links: [link['href'] for link in links if link['rel'] == 'target-tiles'][0]).map(lambda x: x + '&exp=bincat:0|39039e')
buildings['b_source_quad'] = buildings['links'].map(lambda links: [link['href'] for link in links if link['rel'] == 'source-quad'][0])
buildings['buildings_quad'] = buildings['links'].map(lambda links: [link['href'] for link in links if link['rel'] == 'target-quad'][0])

# drop links column
buildings.drop(labels=['links'], axis=1, inplace=True)

# change column names
buildings.rename(columns = {'properties.observed': 'date'}, inplace=True)
buildings['date'] = buildings['date'].map(lambda x: x.split('T')[0])
buildings.head()

Now, let's do the same as above for our Roads Segmentation results.

In [ ]:
roads = pd.json_normalize(roads_results['features']).loc[:, ['links', 'properties.observed']]

# extract links 
roads['source_tiles'] = roads['links'].map(lambda links: [link['href'] for link in links if link['rel'] == 'source-tiles'][0])
roads['roads_tiles'] = roads['links'].map(lambda links: [link['href'] for link in links if link['rel'] == 'target-tiles'][0]).map(lambda x: x + '&exp=bincat:0|d65a45')
roads['r_source_quad'] = roads['links'].map(lambda links: [link['href'] for link in links if link['rel'] == 'source-quad'][0])
roads['roads_quad'] = roads['links'].map(lambda links: [link['href'] for link in links if link['rel'] == 'target-quad'][0])

# drop links column
roads.drop(labels=['links'], axis=1, inplace=True)

# change column names
roads.rename(columns = {'properties.observed': 'date'}, inplace=True)
roads['date'] = roads['date'].map(lambda x: x.split('T')[0])

roads.head()

Next, let's combine our roads and buildings results into a single feature dataframe.

In [ ]:
rnb = buildings.merge(roads, on=['date', 'source_tiles'])
rnb.head()

Finally, let's create separate dataframes for tiles and quads. We'll use the tiles to visualize our AOI in a few web maps, and the quads for a later analysis.

In [ ]:
tiles = rnb.loc[:, ['date', 'source_tiles', 'buildings_tiles', 'roads_tiles']].drop_duplicates()

# Sort dataframe by time
tiles = tiles.sort_values(by='date').reset_index(drop=True)
tiles.head()

Quads dataframe

In [ ]:
quads = rnb.loc[:, ['date', 'r_source_quad', 'b_source_quad', 'roads_quad', 'buildings_quad']]

# Sort dataframe by time
quads.sort_values(by='date', inplace=True)
quads.reset_index(drop=True, inplace=True)
quads.head()

## 6. Segmentation Results

First, let's take a look at our results for the first date in our subscription.

In [ ]:
times = tiles['date'].unique()
print('First date: {}'.format(times[0]))

In [ ]:
m = Map(center=map_center, zoom=13)

building_mask = LocalTileLayer(
    path=tiles.loc[0, 'buildings_tiles'], 
    name=f"Buildings: {tiles.loc[0, 'date']}"
)

road_mask = LocalTileLayer(
    path=tiles.loc[0, 'roads_tiles'], 
    name=f"Roads: {tiles.loc[0, 'date']}"
)

basemap = LocalTileLayer(
    path=tiles.loc[0, 'source_tiles'], 
    name='Source image'
)

m.add_layer(basemap)
m.add_layer(road_mask)
m.add_layer(building_mask)

m.add_control(LayersControl(position='topright'))

m

**Split Map**

Using the `SplitMapControl`, we can easily swipe between pre- and post-flood imagery and building overlays. 

In [ ]:
m = Map(center=map_center, zoom=13)

# first time-point
building_mask_1 = LocalTileLayer(
    path=tiles.loc[16, 'buildings_tiles'], name=f"Buildings: {tiles.loc[16, 'date']}")

road_mask_1 = LocalTileLayer(
    path=tiles.loc[16, 'roads_tiles'], name=f"Roads: {tiles.loc[16, 'date']}")

basemap_1 = LocalTileLayer(
    path=tiles.loc[16, 'source_tiles'], name=tiles.loc[17, 'date'])


# add second time series
building_mask_2 = LocalTileLayer(
    path=tiles.loc[18, 'buildings_tiles'], name=f"Buildings: {tiles.loc[18, 'date']}")

road_mask_2 = LocalTileLayer(
    path=tiles.loc[18, 'roads_tiles'], name=f"Roads: {tiles.loc[18, 'date']}")

basemap_2 = LocalTileLayer(
    path=tiles.loc[18, 'source_tiles'], name=tiles.loc[18, 'date'])

# add layers
m.add_layer(road_mask_1)
m.add_layer(building_mask_1)

m.add_layer(road_mask_2)
m.add_layer(building_mask_2)

splitter = SplitMapControl(left_layer=[building_mask_1, road_mask_1, basemap_1], 
                           right_layer=[building_mask_2, road_mask_2, basemap_2])

# add controls
m.add_control(LayersControl(position='topright'))
m.add_control(splitter)

m

**Date Slider**

We can toggle through our time points with greater granularity using a Date Slider widget.

In [ ]:
m = Map(center=map_center, zoom=13)

date_slider = SelectionSlider(description='Time:', options=times)

current_date = '2017-07-01'

# create and add initial layers
source_layer=LocalTileLayer(path=tiles.loc[tiles['date'] == current_date, 'source_tiles'].iloc[0])
roads_layer=LocalTileLayer(path=tiles.loc[tiles['date'] == current_date, 'roads_tiles'].iloc[0])
buildings_layer=LocalTileLayer(path=tiles.loc[tiles['date'] == current_date, 'buildings_tiles'].iloc[0])

m.add_layer(source_layer)
m.add_layer(roads_layer)
m.add_layer(buildings_layer)

def get_source_url(change):
    global tiles
    source_url = tiles.loc[tiles['date'] == change, 'source_tiles'].iloc[0]
    return source_url

def get_road_url(change):
    global tiles
    roads_url = tiles.loc[tiles['date'] == change, 'roads_tiles'].iloc[0]
    return roads_url

def get_building_url(change):
    global tiles
    buildings_url = tiles.loc[tiles['date'] == change, 'buildings_tiles'].iloc[0]
    return buildings_url

def display_tiles(change):
    global source_layer
    global roads_layer
    global buildings_layer
    global current_date
    
    if current_date != date_slider.value:
        current_date = date_slider.value
    
   # update source imagery
    source_layer = LocalTileLayer(path=get_source_url(current_date))
    
    # update roads mask
    roads_layer= LocalTileLayer(path=get_road_url(current_date))
    
    # update buildings mask
    buildings_layer = LocalTileLayer(path=get_building_url(current_date))
    
    # add new layers
    m.add_layer(source_layer)
    m.add_layer(roads_layer)
    m.add_layer(buildings_layer)

# link 
date_slider.observe(display_tiles, 'value')

**Display Map**

In [ ]:
VBox([date_slider, m])

## 7. Quantifying Change
_Calculating Buildings and Roads Pixels_

The above visualizations allow for a great qualitative analysis of our AOI, with the roads and buildings masks drawing our attention to the changing features over time. Now, let's use some raster tools to quantitatively measure the changes.

**Specify New AOI**

We'll use a condensed polygon centered around the region of development and flooding we observed above.

In [ ]:
new_aoi = {'type': 'Polygon',
           'coordinates': [[[40.746897, 36.270941],
                            [40.746897, 36.294431],
                            [40.790485, 36.294431],
                            [40.790485, 36.270941],
                            [40.746897, 36.270941]]]
          }

**Note**: Coordinate Reference Systems

Leaflet Maps and Planet's Tile Services/ Quads use a different coordinate reference system. So, we'll need to transform the above geometry to properly align our AOI and imagery.

   - Leaflet CRS: `EPSG 4326`
   - Planet CRS: `EPSG 3857`

In [ ]:
import rasterio as rio
from rasterio.warp import transform_geom
from shapely.geometry import shape
from numpy import count_nonzero

In [ ]:
# transform AOI
transformed_aoi = rio.warp.transform_geom(src_crs='EPSG:4326', dst_crs='EPSG:3857', geom=new_aoi, precision=6)

pprint(transformed_aoi)

**Helper functions**

In [ ]:
def get_download_url(quad_url, auth):
    """
    utility function to get the target-quad download url
    
    This enables reading Planet Quads using Rasterio without local download!
    """ 
    resp = requests.get(quad_url, 
                        auth=auth,
                        allow_redirects=False)
    
    assert resp.status_code == 302
    
    return resp.headers['Location']

def get_geometry_bounds(aoi):
    """
    Converts GeoJSON-like feature to Shapely geometry object.
    
    Returns bounds of object
    """
    geo_to_shape = shape(aoi)
    
    xmin, ymin, xmax, ymax = geo_to_shape.bounds
    
    return xmin, ymin, xmax, ymax

def read_window(dataset, xmin, ymin, xmax, ymax):
    """
    Performs a windowed read of a GeoTiff using the geometry bounds of an AOI
    
    Returns the window as a 1 dimensional numpy array.
    """    
    windarray = dataset.read(
                    indexes=1, # only reads the binary segmentation mask band
                    window=rio.windows.from_bounds(
                        xmin, ymin, xmax, ymax,
                        transform=dataset.transform))

    return windarray

def get_pixel_counts(windarray):
    """
    Calculates the sum of non-zero (e.g. roads, buildings) pixels in an array
    """
    return count_nonzero(windarray)

As a proof of concept, let's read in one Quad to get a sense of it's shape.

In [ ]:
first_quad_url = get_download_url(quads.loc[0, 'roads_quad'], auth=BASIC_AUTH)

ex_quad =  rio.open(first_quad_url)
print(ex_quad.read().shape)

As per the [docs](https://developers.planet.com/docs/analytics/apiresponse/#collections-for-raster-results), raster results are in the form of a two band GEOTIFF. The first band contains the binary mask data (this is what we are interested in!). The second band represents valid or invalid source imagery pixels.

Now, let's read all of our Quads and store their pixel counts in a `pixel_counts` dictionary.

Our AOI intersects two source quads, and the segmentation results overlay those quads for each of our time points. So, we'll be reading data from four quads for every point in our time series.

In [ ]:
from tqdm import tqdm

In [ ]:
pixel_counts = {time: {'building_px': 0, 'road_px':0, 'total_px':0} for time in times}

# Get bounds of our AOI
xmin, ymin, xmax, ymax = get_geometry_bounds(transformed_aoi)

# this will take a little while...
for time in tqdm(times):
    building_pixels = 0
    roads_pixels = 0

    for idx, row in quads.loc[quads['date'] == time, :].iterrows():
        
        # get quad items urls
        buildings_quad = row['buildings_quad']
        roads_quad = row['roads_quad']
        
        # get quad download urls
        buildings_url = get_download_url(buildings_quad, auth=BASIC_AUTH)
        roads_url = get_download_url(roads_quad, auth=BASIC_AUTH)
        
        # read quad segmentation mask band
        buildings_data = rio.open(buildings_url)
        roads_data = rio.open(roads_url)
        
        buildings_window = read_window(buildings_data, xmin, ymin, xmax, ymax)
        roads_window = read_window(roads_data, xmin, ymin, xmax, ymax)
        
        # count buildings and road pixels
        building_count = count_nonzero(buildings_window)
        roads_count = count_nonzero(roads_window)
        
        # store in dictionary under time key
        building_pixels += building_count
        roads_pixels += roads_count
    
    pixel_counts[time]['building_px'] = building_pixels
    pixel_counts[time]['road_px'] = roads_pixels
    pixel_counts[time]['total_px'] = building_pixels + roads_pixels

Converting `pixel_counts` to a dataframe will make plotting a breeze!

In [ ]:
pixel_counts = pd.DataFrame(data=pixel_counts.values(), index=pixel_counts.keys())
pixel_counts.head()

# 8. Line Charts

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plot building pixels over time

plt.figure(figsize=(10, 5))
plt.title('Building Pixels over Time', fontdict={'size':18})

plt.vlines(x=[idx for idx, time in enumerate(times) if time == '2017-09-01'][0], 
           ymin=pixel_counts['building_px'].min(), 
           ymax=pixel_counts['building_px'].max(), 
           colors='r', linestyles='dashed', alpha=0.5)

plt.vlines(x=[idx for idx, time in enumerate(times) if time == '2018-11-01'][0], 
           ymin=pixel_counts['building_px'].min(), 
           ymax=pixel_counts['building_px'].max(), 
           colors='r', linestyles='dashed', alpha=0.5)

pixel_counts['building_px'].plot();

Looking at the above plot for building pixels, we can broadly quantify the change in buildings in our AOI over the time of our subscription. As indicated on the plot, we see a sudden spike in buildings from September to Novemeber 2017, as the IDP camp settles on the same penninsula. Then - despite some noise in the pixel counts - we can detect the drop in buildings at the end of 2018 as severe flooding alters the camp and surrounding landscape.

Next, let's look at the pattern for roads.

In [ ]:
plt.figure(figsize=(10, 5))
plt.title('Road Pixels over Time', fontdict={'size':18})

plt.vlines(x=[idx for idx, time in enumerate(times) if time == '2017-09-01'][0], 
           ymin=pixel_counts['road_px'].min(), 
           ymax=pixel_counts['road_px'].max(), 
           colors='r', linestyles='dashed', alpha=0.5)

plt.vlines(x=[idx for idx, time in enumerate(times) if time == '2018-11-01'][0], 
           ymin=pixel_counts['road_px'].min(), 
           ymax=pixel_counts['road_px'].max(), 
           colors='r', linestyles='dashed', alpha=0.5)

pixel_counts['road_px'].plot();

Our plot of road pixels over time shows a different trend. While also prone to some month-to-month noise, we can see early that the high number of road pixels is consistent with the construction of the roads scaffolding as early as August, 2017. After the flooding, we see a slight change in road pixels, too. However, this coincides with new road construction in an adjacent camp in our AOI. 

Finally, we can visualize the count of combined roads and buildings pixels.

In [ ]:
plt.figure(figsize=(10, 5))

plt.title('Roads & Buildings Pixels over Time', fontdict={'size':18})

plt.vlines(x=[idx for idx, time in enumerate(times) if time == '2017-09-01'][0], 
           ymin=pixel_counts['total_px'].min(), 
           ymax=pixel_counts['total_px'].max(), 
           colors='r', linestyles='dashed', alpha=0.5)

plt.vlines(x=[idx for idx, time in enumerate(times) if time == '2018-11-01'][0], 
           ymin=pixel_counts['total_px'].min(), 
           ymax=pixel_counts['total_px'].max(), 
           colors='r', linestyles='dashed', alpha=0.5)

pixel_counts['total_px'].plot();

Looking at the line chart for combined roads and buildings pixels, we can observe the overall trend in the development and change to this IDP camp. With this simple method of quantifying change, we can readily pick out specific time points with broader changes from a large time range. 

With Planet Analytic Feeds, change can be observed in near real-time, enabling enhanced monitoring capabilities, insights at the speed of change, and data-driven decisions.